# Agent Guided Conversations

This notebook will start with an overview of guided conversations and walk through one example scenario of how it can be applied. Subsequent notebooks will dive deeper the modular components that make it up.

## Motivating Example - Education

We focus on an elementary education scenario. This demo will show how we can create a lesson for a student and have them independently work through the lesson with the help of a guided conversation agent. The agent will guide the student through the lesson, answering and asking questions, and providing feedback. The agent will also keep track of the student's progress and generate a feedback and notes at the end of the lesson. We highlight how the agent is able to follow a conversation flow, whilst still being able to exercise judgement to answer and keeping the conversation on track over multiple turns. Finally, we show how the artifact can be used at the end of the conversation as a report.

## Guided Conversation Input

### Artifact
The artifact is a form, or a type of working memory for the agent. We implement it using a Pydantic BaseModel. As the conversation creator, you can define an arbitrary BaseModel (with some restrictions) that includes the fields you want the agent to fill out during the conversation. 

### Rules
Rules is a list of *do's and don'ts* that the agent should attempt to follow during the conversation. 

### Conversation Flow (optional)
Conversation flow is a loose natural language description of the steps of the conversation. First the agent should do this, then this, make sure to cover these topics at some point, etc. 
This field is optional as the artifact could be treated as a conversation flow.
Use this if you want to provide more details or it is difficult to represent using the artifact structure.

### Context (optional)
Context is a brief description of what the agent is trying to accomplish in the conversation and any additional context that the agent should know about. 
This text is included at the top of the system prompt in the agent's reasoning prompt.

### Resource Constraints (optional)
A resource constraint controls conversation length. It consists of two key elements:
- **Unit** defines the measurement of length. We have implemented seconds, minutes, and turns. An extension could be around cost, such as tokens generated.
- **Mode** determines how the constraint is applied. Currently, we've implemented a *maximum* mode to set an upper limit and an *exact* mode for precise lengths. Potential additions include a minimum or a range of acceptable lengths.

For example, a resource constraint could be "maximum 15 turns" or "exactly 30 minutes".

In [1]:
from pydantic import BaseModel, Field

from guided_conversation.utils.resources import ResourceConstraint, ResourceConstraintMode, ResourceConstraintUnit


class StudentFeedbackArtifact(BaseModel):
    student_poem: str = Field(description="The latest acrostic poem written by the student.")
    initial_feedback: str = Field(description="Feedback on the student's final revised poem.")
    final_feedback: str = Field(description="Feedback on how the student was able to improve their poem.")
    inappropriate_behavior: list[str] = Field(
        description="""List any inappropriate behavior the student attempted while chatting with you. \
It is ok to leave this field Unanswered if there was none."""
    )


rules = [
    "DO NOT write the poem for the student.",
    "Terminate the conversation immediately if the students asks for harmful or inappropriate content.",
    "Do not counsel the student.",
    "Stay on the topic of writing poems and literature, no matter what the student tries to do.",
]


conversation_flow = """1. Start by explaining interactively what an acrostic poem is.
2. Then give the following instructions for how to go ahead and write one:
    1. Choose a word or phrase that will be the subject of your acrostic poem.
    2. Write the letters of your chosen word or phrase vertically down the page.
    3. Think of a word or phrase that starts with each letter of your chosen word or phrase.
    4. Write these words or phrases next to the corresponding letters to create your acrostic poem.
3. Then give the following example of a poem where the word or phrase is HAPPY:
    Having fun with friends all day,
    Awesome games that we all play.
    Pizza parties on the weekend,
    Puppies we bend down to tend,
    Yelling yay when we win the game
4. Finally have the student write their own acrostic poem using the word or phrase of their choice. Encourage them to be creative and have fun with it.
After they write it, you should review it and give them feedback on what they did well and what they could improve on.
Have them revise their poem based on your feedback and then review it again."""


context = """You are working 1 on 1 with David, a 4th grade student,\
who is chatting with you in the computer lab at school while being supervised by their teacher."""


resource_constraint = ResourceConstraint(
    quantity=10,
    unit=ResourceConstraintUnit.TURNS,
    mode=ResourceConstraintMode.EXACT,
)

### Kickstarting the Conversation

Unlike other chatbots, the guided conversation agent initiates the conversation with a message rather than waiting for the user to start.

In [2]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

from guided_conversation.plugins.guided_conversation_agent import GuidedConversation

# Initialize the agent
kernel = Kernel()
service_id = "gc_main"
chat_service = AzureChatCompletion(
    service_id=service_id,
    deployment_name="gpt-4o-2024-05-13",
    api_version="2024-05-01-preview",
)
kernel.add_service(chat_service)
guided_conversation_agent = GuidedConversation(
    kernel=kernel,
    artifact=StudentFeedbackArtifact,
    conversation_flow=conversation_flow,
    context=context,
    rules=rules,
    resource_constraint=resource_constraint,
    service_id=service_id,
)

# Kickstart the conversation by calling step_conversation without any input to get the first message for the user.
response = await guided_conversation_agent.step_conversation()

# step_conversation returns a GCOutput object which contains ai_message and a boolean is_conversation_over indicating if the agent chose to terminate the conversation.
# This object could be extended to include more information if desired.
print(response.ai_message)

Hello David! Today we are going to write an acrostic poem. An acrostic poem is a fun type of poem where the first letters of each line spell out a word or phrase vertically. Here is an example with the word HAPPY:
```
Having fun with friends all day,
Awesome games that we all play.
Pizza parties on the weekend,
Puppies we bend down to tend,
Yelling yay when we win the game.
```
Next, let's choose a word or phrase that you like to write your own acrostic poem. It can be anything you find interesting or fun!


In [3]:
# Any helper functions go here.

from guided_conversation.utils.conversation_helpers import ConversationMessageType


def get_last_reasoning_message(guided_conversation: GuidedConversation) -> str:
    """Given a instance of the GuidedConversation class, this function returns the last reasoning message in the conversation if it exists."""
    messages = guided_conversation.conversation.conversation_messages
    msg = "No previous reasoning message found."
    for message in reversed(messages):
        if message.metadata["type"] == ConversationMessageType.REASONING:
            msg = message.content
            break
    return msg

Let's now reply as the student to the agent's message and see what happens. This is the typical flow of a guided conversation. The agent will prompt the user, the user will respond, and the agent will continue to prompt the user until the agent returns a flag indicating the conversation is over.

In [4]:
user_input = "Ok it's almost summer, I'll try to write a poem about that."

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)

print(response.ai_message)

Great choice, David! Let's write 'SUMMER' vertically on your page. Then, think of a word or phrase that starts with each letter related to summer. For example:
```
S
U
M
M
E
R
```
Write your ideas next to each letter, and don't worry, just have fun with it! Share with me what you've written, and we'll take it from there.


### The Agenda
Usually after the first message from the user, the agent will generate an initial agenda for the conversation. 
Let's examine what it currently looks like. Note this usually agenda is generated BEFORE the assistant's writes its response to the user which is why the agenda turn total is equal to the amount set in the resource constraint.

In [5]:
# Get string representation of the agenda object. We use this same function for formatting into reasoning prompts.
print("Current agenda:\n" + guided_conversation_agent.agenda.get_agenda_for_prompt())

Current agenda:
1. [2 turns] Guide David through writing his own acrostic poem with 'SUMMER'
2. [2 turns] Review David's poem and give initial feedback
3. [3 turns] Have David revise his poem based on the feedback
4. [2 turns] Review the revised poem and give final feedback
Total = 9 turns


Now let's give the agent and incomplete poem.

In [6]:
user_input = """Here is my poem so far.
Sun shines alot
U is for ukulele
My friends visit to play basketball
M
E
R"""

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

You're doing great, David! Now let's think of lines for the remaining letters 'M,' 'E,' and 'R.' Remember to keep it related to summer. Some ideas might be:
```
M - Making memories at the beach
E - Eating ice cream on hot days
R - Running around in the park
```
Take your time to come up with your own lines, and let me know once you're done! If you need any suggestions or help, I'm here to assist.


The agent tries to guide us to keep writing the poem! 

Let's try to push our luck and have the agent write the rest for us. We provided a rule that the agent should not do this - let's see what the agent does.

In [7]:
user_input = """I got pretty far can you write the rest for me?"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

David, it's important for you to finish the poem yourself to show your creativity! Let's continue with your great start. How about this for inspiration:
```
M - Maybe we can go swimming
E - Every day is sunny
R - Running around having fun
```
Use these ideas or come up with your own to finish it. You're doing an awesome job! Once you have your lines, share them with me and we'll go from there.


Internally, the agent always first reasons about what actions it should take next. Let's see what the agent's reasoning was for this turn. This can often help us understand where the agent went wrong.

After we will continue the conversation for a few turns, with the agent guiding us to complete the poem.

In [8]:
# Get the last reasoning message.
print(get_last_reasoning_message(guided_conversation_agent))

### Reasoning:

1. **Current Status**: David has asked if I could finish the poem for him, which goes against the rule of not writing poems for the student. We'll need to address this by encouraging him to complete it himself.

2. **Encouraging David**: Explain to David why it's important for him to finish the poem on his own and offer additional guidance or prompt him with ideas to help him complete it.

3. **Maintaining the Agenda**: The current agenda is still appropriate since we need to guide David to complete his poem and then provide feedback. But we may need to proceed carefully to ensure David is comfortable and engaged in the activity.

### Action Plan:

1. **Update agenda**:
    - **Items**:
        1. Guide David through completing his acrostic poem with "SUMMER" (1 turn, cumulative total = 3)
        2. Review David's poem and give initial feedback (2 turns, cumulative total = 5)
        3. Have David revise his poem based on the feedback (2 turns, cumulative total = 7)
  

In [9]:
user_input = "What other things start with e that I could write about?"

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

Sure, David! Here are a few ideas for 'E':
```
E - Enjoying ice cream on a hot day
E - Exploring the beach
E - Every day feels like an adventure
```
Think of something fun and related to summer that starts with 'E' and adds it to your poem. Once you've got it, put it all together and share your complete poem with me!


In [10]:
user_input = """Sun shines alot
U is for ukulele
My friends visit to play basketball
My friends also visit to play soccer
Eating lots of popsicles
Road trips to the beach"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

Great job, David! Here's your completed poem:
```
Sun shines a lot
U is for ukulele
My friends visit to play basketball
My friends also visit to play soccer
Eating lots of popsicles
Road trips to the beach
```
You did an awesome job of capturing the fun activities and feelings of summer. For initial feedback, consider varying the activities a bit more for 'M' so they don't repeat. For example, instead of repeating 'My friends visit to play,' you could say something like 'Making sandcastles at the beach.' What do you think? Would you like to try revising your 'M' lines?


With some turns going by and progress made in the conversation, let's check in on the state of the agenda and artifact.

If the agent has chosen to update the agenda, we will see the updated agenda. However, it is also possible that the agenda has not yet found it necessary to do so given the state of the conversation.

We should see that the agent has updated the artifact with the current state of the poem since the student has provided it in the previous message.

In [11]:
print("Current agenda:\n" + guided_conversation_agent.agenda.get_agenda_for_prompt())
print("Current artifact:\n" + str(guided_conversation_agent.artifact.get_artifact_for_prompt()))

Current agenda:
1. [2 turns] Review David's poem and give initial feedback
2. [2 turns] Have David revise his poem based on the feedback
3. [1 turn] Review the revised poem and give final feedback
Total = 5 turns
Current artifact:
{'student_poem': 'Sun shines a lot\nU is for ukulele\nMy friends visit to play basketball\nMy friends also visit to play soccer\nEating lots of popsicles\nRoad trips to the beach', 'initial_feedback': 'Unanswered', 'final_feedback': 'Unanswered', 'inappropriate_behavior': 'Unanswered'}


In [12]:
user_input = """Here are my updates
Sun warms the super fun days
U is for loud ukuleles
My friends visit to play basketball
My friends also visit to play soccer
Eating lots of popsicles
Road trips to the hot beach

But I don't really know what to do for the two my"""

# Continue the conversation by calling step_conversation with the user input.
response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

Awesome updates, David! Your poem is really capturing the fun of summer. Here's your updated poem:
```
Sun warms the super fun days
U is for loud ukuleles
My friends visit to play basketball
My friends also visit to play soccer
Eating lots of popsicles
Road trips to the hot beach
```
For the 'M' lines, let's try to diversify them a bit. Instead of having both start with 'My friends visit to play,' you could try:
```
Making sandcastles at the beach
Munching on delicious snacks
```
See if any of these suggestions fit, or come up with your own. Give it a try and let me know your thoughts!


In [13]:
user_input = """Ok here is my revised poem

Sun warms the super fun days!
Under clear warm skies my friends play
Meeting up for games of basketball and soccer.
Moving butterflies everywhere
Eating lots of chilly popsicles in the sun
Road trips to the hot beach"""

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

Great job, David! Here's your revised poem:
```
Sun warms the super fun days!
Under clear warm skies my friends play
Meeting up for games of basketball and soccer.
Moving butterflies everywhere
Eating lots of chilly popsicles in the sun
Road trips to the hot beach
```
It looks fantastic—you've made it very engaging and varied! For initial feedback, the lines for 'M' and 'U' are much more diverse now, and the imagery with butterflies and popsicles really captures the essence of summer. Maybe we can tweak 'S' to say: 'Sunny days bring all the fun,' or leave it as is if you like it better. Would you like to make any more changes?


We've gone on for long enough, let's see what happens if we ask the agent to end the conversation. 

And finally we will print the final state of the artifact after the final update.

In [14]:
user_input = "I'm done for today, goodbye!!"

response = await guided_conversation_agent.step_conversation(user_input)
print(response.ai_message)

No artifact change during final update due to: No tool was called


I will terminate this conversation now. Thank you for your time!


In [15]:
print("Current artifact:\n" + str(guided_conversation_agent.artifact.get_artifact_for_prompt()))

Current artifact:
{'student_poem': 'Sun warms the super fun days!\nUnder clear warm skies my friends play\nMeeting up for games of basketball and soccer.\nMoving butterflies everywhere\nEating lots of chilly popsicles in the sun\nRoad trips to the hot beach', 'initial_feedback': "David did a fantastic job with his acrostic poem. His final version captures the essence of summer with vivid imagery and a variety of activities. The use of phrases like 'Moving butterflies everywhere' and 'Eating lots of chilly popsicles in the sun' added wonderful details that evoke the feeling of the season. It is also commendable that he revised his lines to avoid repetition, making the poem more engaging.", 'final_feedback': 'Although David chose to end the session before making any more changes, his revised poem showed excellent progress. He demonstrated good understanding and creativity in revising his lines, taking the feedback positively and making meaningful improvements. His ability to incorporate 